In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pylab as plt
from dataset.mnist import load_mnist

# 오차제곱합(손실 함수), 오차제곱합 함수를 거쳐 나온 값이 작을 수록(0에 인접할 수록) 정답일 확률이 높다
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

# 교차 엔트로피 오차(손실 함수), 정답일 때의 출력이 전체 값을 정하게 되며 함수를 거쳐 나온 값이 작을 수록(0에 인접할 수록) 정답일 확률이 높다.
def cross_entropy_error(y, t):
    # y는 신경망의 출력, t는 정답 레이블
    """
    일반 교차 엔트로피 오차 함수
    delta = 1e-7 # 아주 작은 값
    return -np.sum(t * np.log(y + delta)) # np.log() 함수에 0입 입력하면 -inf(마이너스 무한대)가 나오므로 아주작은 값 delta를 더한 것
    """
    # 배치 데이터(미니배치 학습 같을 때 쓰이는 입력 데이터의 묶음 데이터)를 지원하는 교차 엔트로피 오차 함수
    if y.ndim == 1: # 신경망의 출력이 1차원(데이터 하나당 교차 엔트로피 오차 구하는 경우)이라면 reshape 함수로 데이터의 형상 바꾸기
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    # 정답 레이블이 원-핫 인코딩이 아니라 숫자 레이블 같이 주어졌을 때는 return -np.sum(np.log(y[np.arrange(batch_size), t] + 1e-7))/batch_size 로 리턴값 넘겨준다.
    return -np.sum(t * np.log(y + 1e-7))/batch_size # 원-핫 인코딩일 때는 t가 0인 원소는 교차 엔트로피 오차도 0이라 그 계산은 무시한다.

# 수치 미분 함수, 아주 작은 차분으로 미분 하는 것을 '수치 미분'이라 한다. -> 일반 수학시간에 배우는 미분을 근사치로 계산하는 방법
def numerical_diff(f ,x):
    h = 1e-4 # 0.0001, 이 값을 이용하면 반올림 오차 문제(작은 값이 생략되어 결과에 오차가 생기는 것)를 예방하면서 좋은결과를 얻는다고 알려짐
    return (f(x+h)-f(x-h))/(2*h) # 중심 차분 or 중앙 차분 방법, x를 중심으로 h만큼 차분 계산한다는 것 / 이 외에도 x+h와 x 같은 전방 차분도 있다.

"""
4.2 손실 함수

정확도는 미분 값이 대부분의 장소에서 0이 되어 매개변수를 갱신할 수 없기 때문에
손실 함수를 지표로 삼아서 가중치 매개변수에 대한 손실함수의 미분이 음수면 가중치 매개변수를 양의 방향으로, 양수면 음의방향으로
갱신시켜가며 손실 함수의 값을 줄여가다가 미분 값이 '0'이면 가중치 매개변수의 갱신이 끝난다.
* 미분 = 한순간의 변화량

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True) # one_hot_label은 원핫인코딩의 여부를 묻는 것이다.
print(x_train.shape)
print(t_train.shape)

# ↓ 미니배치 학습, 많은 입력 데이터 중 일부를 추려 전체의 '근사치'로 이용하여 학습하는 것
train_size = x_train.shape[0]
batch_size = 10
# np.random.choice()는 지정한 범위(0 ~ train_size)의 수 중에서 무작위로 원하는 개수(batch_size)만 꺼내는 함수
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
"""
